Die Hodge-Laplace Gleichungen mit Dirichlet Randbedingungen in Vektor Proxy Notation für 1-formen in 3D. Finde $(\mathbf{u}, p) \in \mathbf{V} \times Q$, sodass
$$
\begin{align}
\nabla p  - \nabla \times \nabla \times \mathbf{u} &= \mathbf{f}, \quad &\text{in }\Omega \\
\mathrm{div} \, \mathbf{u} - p &= 0, \quad &\text{in }\Omega, \\
\mathbf{u} &= \mathbf{g}, \quad &\text{auf }\Gamma.
\end{align}
$$
Wobei gilt $\mathbf{g} = \gamma_{\parallel}(\mathbf{g}) +  \mathrm{n} \cdot (\mathbf{g} \cdot \mathrm{n})$. (Zerlegung von g in tangential und normal komponente)

$\gamma_{\parallel}(\mathbf{g}) = \mathrm{n} \times (\mathrm{g} \times \mathrm{n}) $ und $ \gamma_{\mathrm{n}}(\mathbf{g}) = \mathrm{n} \times (\nabla \times \mathrm{n}) $

Wir suchen $(\mathbf{u}_h, p_h) \in V_h \times Q_h$ sodass gilt

$$
\begin{align}
    \int_{\Omega} \nabla p_h \cdot \mathbf{v}_h \, \mathrm{dx} - \int_{\Omega} (\nabla \times \mathbf{u}_h) \cdot (\nabla \times \mathbf{v}_h) \, \mathrm{dx}
    - \int_{\Gamma}\gamma_{n}(\mathbf{u}_h) \cdot \gamma_{\parallel}(\mathbf{v}_h) \, \mathrm{ds} \\[0.25cm]
    - \int_{\Gamma}\gamma_{n}(\mathbf{v}_h) \cdot \gamma_{\parallel}(\mathbf{u}_h) \, \mathrm{ds}
    + \frac{C_{\omega}}{\mathrm{h}} \int_{\Gamma} \gamma_{\parallel}(\mathbf{u}_h) \cdot \gamma_{\parallel}(\mathbf{v}_h) \, \mathrm{ds}
    &= \int_{\Omega} \mathbf{f} \cdot \mathbf{v}_h \, \mathrm{dx}
    - \int_{\Gamma}\gamma_{n}(\mathbf{v}_h) \cdot \gamma_{\parallel}(\mathbf{g}) \, \mathrm{ds},\\[0.25cm]
    &\quad \, + \frac{C_{\omega}}{\mathrm{h}} \int_{\Gamma} \gamma_{\parallel}(\mathbf{g}) \cdot \gamma_{\parallel}(\mathbf{v}_h) \, \mathrm{ds}&\quad \\[0.25cm]
    \int_{\Omega} \mathbf{u}_h \cdot \nabla q_h \, \mathrm{dx}  - \int_{\Omega} p_h \, q_h \, \mathrm{dx}
    &= \int_{\Gamma} (\mathbf{g}\cdot \mathbf{n}) \, q_h \, \mathrm{ds}, \quad \forall (\mathbf{v}_h, q_h) \in V_h \times Q_h.
\end{align}
$$

Wir haben den Symmetrisierungsterm $- \int_{\Gamma}\gamma_{n}(\mathbf{u}_h) \cdot \gamma_{\parallel}(\mathbf{v}_h) \, \mathrm{ds}$ und den Nitsche penalty Term $ \frac{C_{\omega}}{\mathrm{h}} \int_{\Gamma} \gamma_{\parallel}(\mathbf{u}_h) \cdot \gamma_{\parallel}(\mathbf{v}_h) \, \mathrm{ds}$ addiert und die Rechte Seite der Gleichung entsprechend angepasst. Im Nitsche Term wird die Tangetialkomponente der Dirichlet Randbedingung approximiert (oder abweichung davon "penalisiert"), wobei die normalkomponente im Randintegral von der partiellen Integration natürlich eingebettet wird.

In [2]:
# Importiere Sachen

from ngsolve import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.csg import *

from ngsolve.comp import ProxyFunction
from ngsolve.fem import CoefficientFunction

ProxyFunction.__or__ = lambda a, b: InnerProduct(a, b)
CoefficientFunction.__or__ = lambda a, b: InnerProduct(a, b)

In [3]:
# Funktionen um Hodge-Laplace von g zu berechnen
coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """
    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))
    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """
    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """
    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

In [4]:
# Funktion für Geometrie

def createUnitBrickGeometry(h_init, refineSteps = None):
    unitBrick = OrthoBrick( Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5) )
    geo = CSGeometry()
    geo.Add (unitBrick)
    mesh = geo.GenerateMesh(maxh=h_init)
    if (refineSteps != None):
        for i in range(refineSteps):
            mesh.Refine()
        return Mesh(mesh)
    else:
        return Mesh(mesh)

In [5]:
def hodgeLaplace1Forms(mesh,
                       g,
                       order = 1,
                       C_w = 1):
    
    dS = dx(skeleton  = True, element_vb = BND, definedon=mesh.Boundaries(".*"))
    ProxyFunction.__or__ = lambda a, b: InnerProduct(a, b)
    CoefficientFunction.__or__ = lambda a, b: InnerProduct(a, b)
    def Gn(u): return Cross(n, curl(u)) #gamma_n operator
    def Gt(u): return Cross(n, Cross(u, n)) #gamma_parallel operator
    
    h_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl)
    h_1 = H1(mesh, order=order)     # For 0-forms, H1 space
    fes = h_curl * h_1
    (u, p), (v, q) = fes.TnT()

    B, F = BilinearForm(fes), LinearForm(fes)

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size

    f = CF(GGrad(GDiv(g)) - GCurl(GCurl(g))) # Hodge Laplace g: Grad Div g - curl curl g

    B += -(curl(u)|curl(v)) * dx
    B += (v|Grad(p)) * dx
    B += (u|Grad(q)) * dx
    B += - (p|q) * dx
    
    B += - (Gt(v)|Gn(u)) * dS
    B += - (Gt(u)|Gn(v)) * dS
    B += - C_w/h * (Gt(u)|Gt(v)) * dS

    F += (f|v) * dx
    
    F +=  - (C_w / h) * (Gt(g)|Gt(v)) * dS
    F +=  - (Gt(g)|Gn(v)) * dS
    F +=  (g*n|q) * dS
    
    with TaskManager():
        B.Assemble(), F.Assemble()
        sol = GridFunction(fes)
        res = F.vec-B.mat * sol.vec
        inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
        sol.vec.data += inv * res

    gf_u , gf_p = sol.components

    p_m =  CF(GDiv(g))
    
    L2_error_u = sqrt(Integrate((gf_u - g)**2, mesh))
    L2_u = sqrt(Integrate((gf_u)**2, mesh))
    L2_error_p = sqrt(Integrate((gf_p - p_m)**2, mesh))
    L2_p = sqrt(Integrate((p_m)**2, mesh))

    return L2_error_u, L2_error_p, L2_u, L2_p, gf_u, gf_p

In [6]:
order = 1
h = 0.1
C_w = 10 #Nitsche penalty parameter

g = CF((x**2 * sin(z) * cos(y), # Hergestellte Lösung
        2 * z**3 * sin(x) * cos(1/3*z),
        -y**2 *-cos(3*z)*sin(x)         ))

mesh = createUnitBrickGeometry(h)

In [7]:
error_u, error_p, L2_u, L2_p,  gf_u, gf_p = hodgeLaplace1Forms(mesh, g, order, C_w)
print("L2 Fehler u: ", error_u)
print("L2 u: ", L2_u)
print("L2 Fehler p: ", error_p)
print("L2 p: ", L2_p)

L2 Fehler u:  0.5643161614640166
L2 u:  0.5674558362855283
L2 Fehler p:  8.810228813591259
L2 p:  0.11840317655252028


In [8]:
Draw(gf_u)
#Draw(gf_p)
#Draw(g, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene